# Dialogues Multi-Agents Argumentatifs

**Navigation**: [← Tweety-7b-Ranking-Probabilistic](Tweety-7b-Ranking-Probabilistic.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-9-Preferences →](Tweety-9-Preferences.ipynb)

---

## Objectifs pédagogiques

1. Comprendre le framework d'agents de TweetyProject
2. Découvrir les protocoles de dialogue argumentatif
3. Simuler des débats entre agents avec échanges structurés
4. Explorer les stratégies argumentatives et la résolution de conflits

## Prérequis

Exécutez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement JVM.

## Modules TweetyProject couverts

- `org.tweetyproject.agents` - Framework agents de base
- `org.tweetyproject.agents.dialogues` - Simulation de dialogues argumentatifs

In [ ]:
# --- Initialisation JVM Tweety + Outils Externes ---
print("--- Verification JVM Tweety ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import pathlib
import shutil

EXTERNAL_TOOLS = {"CLINGO": "", "SPASS": ""}

# Auto-detection Clingo
clingo_paths = [shutil.which("clingo"), shutil.which("clingo.exe"),
    pathlib.Path("ext_tools/clingo/clingo.exe"), pathlib.Path("../ext_tools/clingo/clingo.exe")]
for cp in clingo_paths:
    if cp:
        if isinstance(cp, str): EXTERNAL_TOOLS["CLINGO"] = str(pathlib.Path(cp).parent.resolve()); break
        elif cp.exists(): EXTERNAL_TOOLS["CLINGO"] = str(cp.parent.resolve()); break

# Auto-detection SPASS
spass_paths = [shutil.which("SPASS"), shutil.which("SPASS.exe"),
    pathlib.Path("ext_tools/spass/SPASS.exe"), pathlib.Path("../ext_tools/spass/SPASS.exe")]
for sp in spass_paths:
    if sp:
        if isinstance(sp, str): EXTERNAL_TOOLS["SPASS"] = sp; break
        elif sp.exists(): EXTERNAL_TOOLS["SPASS"] = str(sp.resolve()); break

# === Initialisation JVM ===
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    jdk_portable = pathlib.Path("jdk-17-portable")
    if not jdk_portable.exists(): jdk_portable = pathlib.Path("Argument_Analysis/jdk-17-portable")
    
    if jdk_portable.exists():
        zulu_dirs = list(jdk_portable.glob("zulu*"))
        if zulu_dirs:
            java_home = zulu_dirs[0]
            os.environ["JAVA_HOME"] = str(java_home.resolve())
            print(f"JDK portable trouve: {java_home.name}")
    
    if not os.environ.get("JAVA_HOME"):
        print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")
    else:
        LIB_DIR = pathlib.Path("libs")
        if not LIB_DIR.exists(): LIB_DIR = pathlib.Path("Argument_Analysis/libs")
        
        if LIB_DIR.exists():
            jar_files = list(LIB_DIR.glob("*.jar"))
            if jar_files:
                classpath = os.pathsep.join(str(j.resolve()) for j in jar_files)
                try:
                    jpype.startJVM(classpath=[classpath])
                    print(f"JVM demarree avec {len(jar_files)} JARs.")
                    jvm_ready = True
                except Exception as e: print(f"Erreur demarrage JVM: {e}")

if jvm_ready:
    print("JVM prete pour Tweety.")
    tools = [t for t, p in EXTERNAL_TOOLS.items() if p]
    if tools: print(f"Outils externes: {', '.join(tools)}")

## Partie 6 : Agents et Dialogues Argumentatifs

Cette section explore le framework multi-agents de TweetyProject et les protocoles de dialogue argumentatif. Les dialogues permettent de modéliser des échanges structurés entre agents qui défendent des positions contradictoires.

### 6.1 Framework Agents de Base

Le module `org.tweetyproject.agents` fournit les abstractions fondamentales pour la modélisation multi-agents:

- **Agent**: Classe de base représentant un agent autonome
- **Environment**: Environnement dans lequel évoluent les agents
- **Perceivable**: Interface pour les objets perceptibles
- **MultiAgentSystem**: Système regroupant plusieurs agents

Ces composants permettent de construire des simulations où des agents interagissent selon des protocoles définis.

In [ ]:
# --- 6.1 Framework Agents de Base ---
print("\n--- 6.1 Framework Agents de Base ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Exploration du framework agents...")
    agents_imports_ok = False
    try:
        import jpype
        from jpype.types import *
        
        # Imports agents de base
        try:
            from org.tweetyproject.agents import Agent, DummyAgent, MultiAgentSystem, Environment
            print("Imports agents de base reussis.")
            agents_imports_ok = True
        except ImportError as e:
            print(f"Imports agents directs echoues: {e}")
            # Tentative avec JClass
            try:
                Agent = jpype.JClass("org.tweetyproject.agents.Agent")
                DummyAgent = jpype.JClass("org.tweetyproject.agents.DummyAgent")
                MultiAgentSystem = jpype.JClass("org.tweetyproject.agents.MultiAgentSystem")
                print("Imports agents via JClass reussis.")
                agents_imports_ok = True
            except Exception as e2:
                print(f"Echec JClass agents: {e2}")

        if agents_imports_ok:
            print("\n--- Classes disponibles dans org.tweetyproject.agents ---")
            
            # Lister les classes disponibles
            agents_package = jpype.JPackage("org.tweetyproject.agents")
            print("   - Agent (classe abstraite de base)")
            print("   - DummyAgent (agent simple sans comportement)")
            print("   - MultiAgentSystem (systeme multi-agents)")
            print("   - Environment (interface environnement)")
            
            # Créer un agent simple
            print("\n--- Creation d'un agent simple ---")
            try:
                dummy = DummyAgent("AgentTest")
                print(f"   Agent cree: {dummy}")
                print(f"   Nom: {dummy.getName()}")
            except Exception as e_create:
                print(f"   Erreur creation agent: {e_create}")
                print("   Note: La creation d'agents peut necessiter des classes concretes specifiques.")

    except Exception as e_gen:
        print(f"Erreur Python inattendue: {e_gen}")
        import traceback; traceback.print_exc()

### 6.2 Dialogues Argumentatifs

Le module `org.tweetyproject.agents.dialogues` permet de simuler des dialogues structurés où des agents échangent des arguments:

- **DialogueTrace**: Historique des échanges dans un dialogue
- **ExecutableMove**: Action exécutable dans un dialogue (assertion, question, etc.)
- **GroundedSolverProtocol**: Protocole basé sur la sémantique grounded

Les dialogues argumentatifs suivent généralement un protocole définissant:
1. **Les coups autorisés** (moves): assertion, question, challenge, retract
2. **Les règles de tour**: qui peut parler et quand
3. **Les conditions de terminaison**: consensus, impasse, timeout

In [ ]:
# --- 6.2 Dialogues Argumentatifs ---
print("\n--- 6.2 Dialogues Argumentatifs ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Exploration des dialogues argumentatifs...")
    dialogues_imports_ok = False
    try:
        import jpype
        from jpype.types import *
        
        # Imports dialogues
        try:
            # Le module dialogues peut avoir plusieurs sous-packages
            from org.tweetyproject.agents.dialogues import ExecutableExtension
            print("Import ExecutableExtension reussi.")
            dialogues_imports_ok = True
        except ImportError as e:
            print(f"Import direct ExecutableExtension echoue: {e}")
        
        # Tentative d'exploration du package dialogues
        try:
            dialogues_pkg = jpype.JPackage("org.tweetyproject.agents.dialogues")
            print("\n--- Exploration du package agents.dialogues ---")
            
            # Lister les classes connues
            known_classes = [
                "ExecutableExtension",
                "DialogueTrace", 
                "ExecutableMove",
                "ArgumentativeAgent",
                "GroundedSolverProtocol",
                "lotteries.AbstractLotteryAgent",
                "lotteries.LotteryGameSystem",
            ]
            
            print("Classes connues dans agents.dialogues:")
            for cls_name in known_classes:
                try:
                    full_name = f"org.tweetyproject.agents.dialogues.{cls_name}"
                    cls = jpype.JClass(full_name)
                    print(f"   [OK] {cls_name}")
                except Exception:
                    print(f"   [--] {cls_name} (non trouve)")
                    
            dialogues_imports_ok = True
            
        except Exception as e_explore:
            print(f"Erreur exploration package: {e_explore}")

        # --- Exemple de Dialogue Argumentatif Simplifié ---
        if dialogues_imports_ok:
            print("\n--- Exemple Conceptuel de Dialogue ---")
            print("""
Dans un dialogue argumentatif typique:

1. Agent A avance l'argument: "Il pleut, donc je prends un parapluie"
   Move: ASSERT(A, pluie -> parapluie)

2. Agent B conteste: "Mais le bulletin meteo dit beau temps"
   Move: CHALLENGE(B, pluie)
   
3. Agent A defend: "Je vois des nuages noirs"
   Move: ASSERT(A, nuages_noirs -> pluie)
   
4. Agent B accepte ou propose une contre-evidence
   ...

Le protocole determine:
- Qui peut parler (alternance ou libre)
- Quels moves sont valides
- Quand le dialogue se termine

TweetyProject fournit des implementations de ces protocoles,
notamment pour les jeux de loterie argumentative (lotteries).
""")

    except Exception as e_gen:
        print(f"Erreur Python inattendue: {e_gen}")
        import traceback; traceback.print_exc()

### 6.3 Jeux de Loterie Argumentative

Les jeux de loterie argumentative (`agents.dialogues.lotteries`) combinent l'argumentation probabiliste avec les dialogues multi-agents:

- **AbstractLotteryAgent**: Agent participant à un jeu de loterie
- **LotteryGameSystem**: Système gérant le jeu
- **UtilityFunction**: Fonction d'utilité pour évaluer les résultats

Ces jeux permettent de modéliser des situations de négociation où les agents ont des préférences sur les issues possibles du dialogue.

In [ ]:
# --- 6.3 Jeux de Loterie Argumentative ---
print("\n--- 6.3 Jeux de Loterie Argumentative ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Exploration des jeux de loterie...")
    try:
        import jpype
        from jpype.types import *
        
        # Imports nécessaires
        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        from org.tweetyproject.arg.dung.semantics import Semantics
        from org.tweetyproject.arg.prob.lotteries import SubgraphProbabilityFunction, ArgumentationLottery, UtilityFunction
        from org.tweetyproject.arg.dung.divisions import Division
        from org.tweetyproject.arg.dung.semantics import Extension
        
        print("Imports pour loteries reussis.")
        
        # Créer un framework simple
        print("\n--- Creation d'un scenario de loterie ---")
        theory = DungTheory()
        a = Argument("proposition_A"); b = Argument("proposition_B"); c = Argument("evidence_C")
        theory.add(a); theory.add(b); theory.add(c)
        theory.add(Attack(a, b)); theory.add(Attack(b, a)); theory.add(Attack(c, b))
        
        print(f"Framework: {theory}")
        print("""
Scenario:
- Agent 1 supporte proposition_A
- Agent 2 supporte proposition_B  
- evidence_C attaque B, favorisant A
""")
        
        # Distribution de probabilité
        prob_func = SubgraphProbabilityFunction(theory)
        print(f"\nNombre de configurations possibles: {prob_func.size()}")
        
        # Calcul des probabilités d'acceptation
        target_sem = Semantics.GROUNDED_SEMANTICS
        try:
            p_a = prob_func.getAcceptanceProbability(a, target_sem).doubleValue()
            p_b = prob_func.getAcceptanceProbability(b, target_sem).doubleValue()
            p_c = prob_func.getAcceptanceProbability(c, target_sem).doubleValue()
            
            print(f"\nProbabilites d'acceptation (Grounded):")
            print(f"  P(proposition_A accepte) = {p_a:.3f}")
            print(f"  P(proposition_B accepte) = {p_b:.3f}")
            print(f"  P(evidence_C accepte)    = {p_c:.3f}")
            
            print(f"""
Interpretation:
- Si evidence_C est presente, B est attaque et A gagne
- La probabilite que A soit accepte ({p_a:.1%}) > celle de B ({p_b:.1%})
- Un agent rationnel devrait investir pour maintenir C dans le debat
""")
        except Exception as e_prob:
            print(f"Erreur calcul probabilites: {e_prob}")
        
    except ImportError as e:
        print(f"Erreur d'import pour loteries: {e}")
    except jpype.JException as e_java:
        print(f"Erreur Java: {e_java.message()}")
    except Exception as e_gen:
        print(f"Erreur Python inattendue: {e_gen}")
        import traceback; traceback.print_exc()

---

## Résumé

Ce notebook a introduit:
- **Framework Agents**: Classes de base pour la simulation multi-agents
- **Dialogues Argumentatifs**: Protocoles d'échange structurés (moves, traces)
- **Jeux de Loterie**: Combinaison argumentation probabiliste et dialogue

## Limitations et Notes

Le module `agents.dialogues` de TweetyProject est relativement avancé et nécessite souvent:
- Une compréhension des protocoles de dialogue (Walton, Prakken-Sartor)
- L'implémentation de classes concrètes héritant des classes abstraites
- La configuration fine des règles du protocole

Pour une utilisation approfondie, consultez les exemples Java de TweetyProject.

## Prochaines étapes

Le notebook suivant explore le module Preferences pour l'agrégation de préférences et la théorie du vote.

---

**Navigation**: [← Tweety-7b-Ranking-Probabilistic](Tweety-7b-Ranking-Probabilistic.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-9-Preferences →](Tweety-9-Preferences.ipynb)